<font size='5'>期末報告</font><br>

In [16]:
import numpy as np
import pandas as pd
import calendar
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv', engine='python',index_col="ID")
is_canceled=train["is_canceled"]
train_drop=train.drop(["is_canceled","adr","reservation_status","reservation_status_date"],axis=1)
# train_drop=train.drop(["is_canceled","country","adr","reservation_status","reservation_status_date","agent","company"],axis=1)
train_drop=train_drop.fillna(0)
train_drop
# test = pd.read_csv('test.csv', engine='python',index_col="ID")
# train_label = pd.read_csv('train_label.csv', engine='python')
# test_nolabel = pd.read_csv('test_nolabel.csv', engine='python')

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,
0,Resort Hotel,342,2015,July,27,1,0,0,2,0.0,...,C,C,3,No Deposit,0.0,0.0,0,Transient,0,0
1,City Hotel,257,2015,July,27,1,0,2,1,0.0,...,A,A,1,No Deposit,6.0,0.0,0,Transient,0,0
2,City Hotel,257,2015,July,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
3,City Hotel,257,2015,July,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
4,City Hotel,257,2015,July,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,Resort Hotel,19,2017,March,13,31,0,2,2,2.0,...,C,C,0,No Deposit,250.0,0.0,0,Transient,0,1
91527,Resort Hotel,28,2017,March,13,31,0,2,2,0.0,...,A,A,0,No Deposit,0.0,0.0,0,Transient,0,0
91528,Resort Hotel,2,2017,March,13,31,0,1,2,0.0,...,E,F,1,No Deposit,0.0,0.0,0,Transient,0,2


In [3]:
month=["","January","February","March","April","May","June","July","August","September","October","November","December"]
for i in range(len(train_drop)):
    train_drop["arrival_date_month"].iloc[i]=month.index(train_drop["arrival_date_month"].iloc[i])
train_drop

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,
0,Resort Hotel,342,2015,7,27,1,0,0,2,0.0,...,C,C,3,No Deposit,0.0,0.0,0,Transient,0,0
1,City Hotel,257,2015,7,27,1,0,2,1,0.0,...,A,A,1,No Deposit,6.0,0.0,0,Transient,0,0
2,City Hotel,257,2015,7,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
3,City Hotel,257,2015,7,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
4,City Hotel,257,2015,7,27,1,0,2,2,0.0,...,A,A,0,No Deposit,6.0,0.0,0,Transient,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,Resort Hotel,19,2017,3,13,31,0,2,2,2.0,...,C,C,0,No Deposit,250.0,0.0,0,Transient,0,1
91527,Resort Hotel,28,2017,3,13,31,0,2,2,0.0,...,A,A,0,No Deposit,0.0,0.0,0,Transient,0,0
91528,Resort Hotel,2,2017,3,13,31,0,1,2,0.0,...,E,F,1,No Deposit,0.0,0.0,0,Transient,0,2


In [4]:
train_drop_du=train_drop.drop(["country"],axis=1)
train_drop_du=pd.get_dummies(train_drop_du, columns=["hotel","meal","market_segment","distribution_channel",
                                                  "reserved_room_type","assigned_room_type","deposit_type","customer_type"
                                                 ])
train_drop_du.columns

Index(['lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC', 'meal_Undefined',
       'market_segment_Aviation', 'market_segment_Complementary',
       'market_segment_Corporate', 'market_segment_Direct',
       'market_segment_Groups', 'market_segment_Offline TA/TO',
       'market_segment_Online TA', 'market_segment_Undefined',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS', 'distribution_channel_TA/TO',
       'distribution_channel_Undefined', 'reserved

In [5]:
train_drop_du_v2=train_drop_du.drop(["meal_Undefined","market_segment_Undefined","distribution_channel_Undefined",
                                     "reserved_room_type_P","assigned_room_type_P","deposit_type_Refundable",
                                     "customer_type_Transient-Party"],axis=1)
train_drop_du_v2.columns

Index(['lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'meal_SC', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'market_segment_Online TA',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS', 'distribution_channel_TA/TO',
       'reserved_room_type_A', 'reserved_room_type_B', 'reserved_room_type_C',
       'reserved_room_ty

In [6]:
train_drop_du_v3=train_drop_du_v2.drop(["meal_SC","market_segment_Online TA","distribution_channel_TA/TO"],axis=1)
train_drop_du_v3.columns

Index(['lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_B', 'reserved_room_type_C',
       'reserved_room_type_D', 'reserved_room_type_E', 'reserved_room_type_F',
       'reserved_room

In [7]:
train_drop_du_v4=train_drop_du_v3.drop(["reserved_room_type_B","reserved_room_type_C","reserved_room_type_E",
                                        "reserved_room_type_F","reserved_room_type_G","reserved_room_type_H","reserved_room_type_L",
                                       "assigned_room_type_B","assigned_room_type_C","assigned_room_type_E","assigned_room_type_F",
                            'assigned_room_type_G', 'assigned_room_type_H','assigned_room_type_I','assigned_room_type_K',
                                        'assigned_room_type_L'],axis=1)
train_drop_du_v4.columns

Index(['lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'custome

In [8]:
train_drop_du_v5=train_drop_du_v4.drop(['arrival_date_year','arrival_date_day_of_month'],axis=1)
train_drop_du_v5.columns

Index(['lead_time', 'arrival_date_month', 'arrival_date_week_number',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'market_segment_Aviation',
       'market_segment_Complementary', 'market_segment_Corporate',
       'market_segment_Direct', 'market_segment_Groups',
       'market_segment_Offline TA/TO', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer

In [9]:
train_drop_du_v6=train_drop_du_v5.drop(['market_segment_Aviation','market_segment_Complementary','market_segment_Corporate',
                                       'market_segment_Direct', 'market_segment_Groups','market_segment_Offline TA/TO'],axis=1)
train_drop_du_v6.columns

Index(['lead_time', 'arrival_date_month', 'arrival_date_week_number',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient'],
      dtype='object')

In [10]:
room_type=[]
ifno=train_drop[train_drop["reserved_room_type"]!=train_drop["assigned_room_type"]].index
for i in range(len(train_drop)):
    if i in ifno:
        room_type.append(1)
    else:
        room_type.append(0)
train_con=pd.concat([train_drop_du_v6,pd.DataFrame(room_type,columns=["room"])],axis=1)
train_con

,lead_time,arrival_date_month,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,reserved_room_type_A,reserved_room_type_D,assigned_room_type_A,assigned_room_type_D,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient,room
0,342,7,27,0,0,2,0.0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,257,7,27,0,2,1,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
2,257,7,27,0,2,2,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
3,257,7,27,0,2,2,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
4,257,7,27,0,2,2,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,19,3,13,0,2,2,2.0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
91527,28,3,13,0,2,2,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
91528,2,3,13,0,1,2,0.0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
91529,30,3,13,3,7,2,0.0,0,0,0,...,1,0,1,0,1,0,0,0,1,0


In [66]:
train_drop_du_v7=train_con.drop(['arrival_date_month'],axis=1)
train_drop_du_v7.columns

Index(['lead_time', 'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient', 'room'],
      dtype='object')

In [73]:
# train_drop_du_v8=train_drop_du_v7
train_drop_du_v8=train_drop_du_v7.drop(['company'],axis=1)
train_drop_du_v8.columns

Index(['lead_time', 'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient', 'room'],
      dtype='object')

In [74]:
stay_day_type=[]
ifno=train_drop[train_drop["stays_in_weekend_nights"]!=train_drop["stays_in_week_nights"]].index
for i in range(len(train_drop)):
    stay_day_type.append(train_drop["stays_in_weekend_nights"].iloc[i]+train_drop["stays_in_week_nights"].iloc[i])

train_drop_du_v9=pd.concat([train_drop_du_v8,pd.DataFrame(stay_day_type,columns=["stay_days"])],axis=1)
train_drop_du_v9.columns

Index(['lead_time', 'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient', 'room', 'stay_days'],
      dtype='object')

In [75]:
train_drop_du_v10=train_drop_du_v9.drop(['stays_in_weekend_nights','stays_in_week_nights'],axis=1)
train_drop_du_v10.columns

Index(['lead_time', 'arrival_date_week_number', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D', 'assigned_room_type_A',
       'assigned_room_type_D', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient', 'room', 'stay_days'],
      dtype='object')

In [76]:
train_drop_du_v11=train_drop_du_v10.drop(['assigned_room_type_A','assigned_room_type_D'],axis=1)
train_drop_du_v11.columns

Index(['lead_time', 'arrival_date_week_number', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'reserved_room_type_A', 'reserved_room_type_D',
       'deposit_type_No Deposit', 'deposit_type_Non Refund',
       'customer_type_Contract', 'customer_type_Group',
       'customer_type_Transient', 'room', 'stay_days'],
      dtype='object')

In [77]:
train_drop_du_v12=train_drop_du_v11.drop(['reserved_room_type_A','reserved_room_type_D'],axis=1)
train_drop_du_v12.columns

Index(['lead_time', 'arrival_date_week_number', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'meal_BB', 'meal_FB', 'meal_HB', 'distribution_channel_Corporate',
       'distribution_channel_Direct', 'distribution_channel_GDS',
       'deposit_type_No Deposit', 'deposit_type_Non Refund',
       'customer_type_Contract', 'customer_type_Group',
       'customer_type_Transient', 'room', 'stay_days'],
      dtype='object')

In [82]:
train_drop.columns

Index(['hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests'],
      dtype='object')

In [83]:
meal_type=[]
# ifno=train_drop[train_drop["stays_in_weekend_nights"]!=train_drop["stays_in_week_nights"]].index
for i in range(len(train_drop)):
    meal_type.append(train_drop_du_v12["meal_BB"].iloc[i]+train_drop_du_v12["meal_FB"].iloc[i]+train_drop_du_v12["meal_HB"].iloc[i])

train_drop_du_v13=pd.concat([train_drop_du_v12,pd.DataFrame(meal_type,columns=["meal"])],axis=1)
train_drop_du_v13=train_drop_du_v13.drop(['meal_BB','meal_FB','meal_HB'],axis=1)
train_drop_du_v13.columns

Index(['lead_time', 'arrival_date_week_number', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 'agent',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'hotel_City Hotel', 'hotel_Resort Hotel',
       'distribution_channel_Corporate', 'distribution_channel_Direct',
       'distribution_channel_GDS', 'deposit_type_No Deposit',
       'deposit_type_Non Refund', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient', 'room', 'stay_days',
       'meal'],
      dtype='object')

In [14]:
# is_canceled
y=pd.get_dummies(is_canceled)
# train_drop_du.columns
y

,0,1
ID,,
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
91526,0,1
91527,1,0
91528,1,0


In [92]:
import numpy as np  
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import optimizers
from keras.callbacks import EarlyStopping
X_train, X_test, y_train, y_test = train_test_split(train_drop_du_v12, y, test_size=0.2)
# 建立簡單的線性執行的模型
model = Sequential()
# Add Input layer, 隱藏層(hidden layer) 有 256個輸出變數
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(16, activation='relu'))
# Add output layer
model.add(Dense(units=2,  activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy']) 

# 將 training 的 input 資料轉為2維
# X = X.astype('float32') 

#當val_loss不在下降後的的20個epochs停下
early_stopping = EarlyStopping(monitor='loss', patience=50, verbose=2)

# 進行訓練, 訓練過程會存在 train_history 變數中
train_history = model.fit(x=X_train, y=y_train, validation_split=0.1, epochs=2000, batch_size=5000, verbose=2, callbacks=[early_stopping])  



Train on 65901 samples, validate on 7323 samples
Epoch 1/2000
 - 1s - loss: 1.3461 - accuracy: 0.6057 - val_loss: 0.7235 - val_accuracy: 0.6216
Epoch 2/2000
 - 0s - loss: 0.6653 - accuracy: 0.6634 - val_loss: 0.5936 - val_accuracy: 0.7034
Epoch 3/2000
 - 0s - loss: 0.5757 - accuracy: 0.7089 - val_loss: 0.5529 - val_accuracy: 0.7268
Epoch 4/2000
 - 0s - loss: 0.5439 - accuracy: 0.7313 - val_loss: 0.5378 - val_accuracy: 0.7354
Epoch 5/2000
 - 0s - loss: 0.5270 - accuracy: 0.7378 - val_loss: 0.5230 - val_accuracy: 0.7315
Epoch 6/2000
 - 0s - loss: 0.5103 - accuracy: 0.7515 - val_loss: 0.5041 - val_accuracy: 0.7567
Epoch 7/2000
 - 0s - loss: 0.4901 - accuracy: 0.7696 - val_loss: 0.4846 - val_accuracy: 0.7767
Epoch 8/2000
 - 0s - loss: 0.4889 - accuracy: 0.7638 - val_loss: 0.5006 - val_accuracy: 0.7639
Epoch 9/2000
 - 0s - loss: 0.4866 - accuracy: 0.7639 - val_loss: 0.4612 - val_accuracy: 0.7901
Epoch 10/2000
 - 0s - loss: 0.4503 - accuracy: 0.7947 - val_loss: 0.4852 - val_accuracy: 0.7765


In [93]:
comp=model.predict(X_test)
comp=np.array([np.argmax(u) for u in comp])
test_label=np.argmax(np.array(y_test),axis=1)
print(classification_report(test_label, comp))
print(confusion_matrix(test_label, comp))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89     11743
           1       0.81      0.77      0.79      6564

    accuracy                           0.85     18307
   macro avg       0.84      0.84      0.84     18307
weighted avg       0.85      0.85      0.85     18307

[[10564  1179]
 [ 1497  5067]]
